## Import MLcps

In [1]:
#import MLcps
from MLcps import getCPS

#### Create Input dataframe for MLcps

In [2]:
import pandas as pd
metrics_list=[]

#Metrics from SVC model (kernel=rbf)
acc = 0.88811 #accuracy
bacc = 0.86136 #balanced_accuracy
prec = 0.86 #precision
rec = 0.97727 #recall
f1 = 0.91489 #F1
mcc = 0.76677 #Matthews_correlation_coefficient
metrics_list.append([acc,bacc,prec,rec,f1,mcc])

#Metrics from SVC model (kernel=linear)
acc = 0.88811
bacc = 0.87841
prec = 0.90
rec = 0.92045
f1 = 0.91011
mcc = 0.76235
metrics_list.append([acc,bacc,prec,rec,f1,mcc])

#Metrics from KNN
acc = 0.78811
bacc = 0.82841
prec = 0.80
rec = 0.82
f1 = 0.8911
mcc = 0.71565
metrics_list.append([acc,bacc,prec,rec,f1,mcc])

metrics=pd.DataFrame(metrics_list,index=["SVM rbf","SVM linear","KNN"],columns=["accuracy","balanced_accuracy","precision","recall","f1","Matthews_correlation_coefficient"])
metrics


,accuracy,balanced_accuracy,precision,recall,f1,Matthews_correlation_coefficient
SVM rbf,0.88811,0.86136,0.86,0.97727,0.91489,0.76677
SVM linear,0.88811,0.87841,0.90,0.92045,0.91011,0.76235
KNN,0.78811,0.82841,0.80,0.82000,0.89110,0.71565


#### Alternatively, load an example dataframe from MLcps

In [3]:
#read input data (a dataframe) or load an example data
metrics_example=getCPS.sample_metrics()
metrics_example

,Accuracy,Balanced Acc,F1,Recall,Precision,Avg precision,roc_auc
Dummy,0.506,0.512,0.651,0.505,0.917,0.916,0.512
LDA,0.938,0.902,0.965,0.946,0.986,0.995,0.964
GP,0.939,0.963,0.965,0.934,0.999,0.977,0.869
BC,0.983,0.948,0.991,0.990,0.991,0.998,0.987
RF,0.987,0.954,0.993,0.994,0.992,0.999,0.997
ETC,0.988,0.958,0.993,0.994,0.993,1.000,0.997
SVM,0.991,0.983,0.995,0.993,0.998,1.000,0.998
LR,0.994,0.989,0.997,0.995,0.998,1.000,0.998


## Calculate MLcps (Machine Learning cumulative performance score)

In [4]:
#calculate Machine Learning cumulative performance score
cpsScore=getCPS.calculate(metrics)
cpsScore

,Algorithms,Score
1,KNN,1.690497
2,SVM linear,1.995185
3,SVM rbf,2.001725


In [5]:
#calculate Machine Learning cumulative performance score
cpsScore=getCPS.calculate(metrics_example)
cpsScore

,Algorithms,Score
1,Dummy,1.154085
2,GP,2.466270
3,LDA,2.504454
4,BC,2.649548
5,RF,2.671151
6,ETC,2.676568
7,SVM,2.703692
8,LR,2.713790


## Plot MLcps

In [6]:
import plotly.express as px
from plotly.offline import plot
import plotly.io as pio
pio.renderers.default = 'iframe' 

fig = px.bar(cpsScore, x='Score', y='Algorithms',color='Score',labels={'MLcps Score'},
             width=700,height=1000,text_auto=True)

fig.update_xaxes(title_text="MLcps")
fig

## MLcps for GridSearch object
##### Calculate MLcps using the mean test score of all the metrics available in the given GridSearch object and return an updated GridSearch object. Returned GridSearch object contains mean_test_MLcps and rank_test_MLcps arrays, which can be used to rank the models similar to any other metric.

In [7]:
#read GridSearch object or load it from package
gsObj=getCPS.sample_GridSearch_Object()

#calculate Machine Learning cumulative performance score
gsObj_updated=getCPS.calculate(gsObj)
gsObj_updated


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=3, random_state=10),
             estimator=Pipeline(steps=[('combineSampling',
                                        SMOTETomek(random_state=10)),
                                       ('scaler', StandardScaler()),
                                       ('feature_sel', SelectKBest()),
                                       ('m', LinearSVC(random_state=10))]),
             n_jobs=-1, param_grid={'feature_sel__k': [200, 400, 1000]},
             refit='average_precision', return_train_score=True,
             scoring=['accuracy', 'balanced_accuracy', 'f1', 'recall',
                      'precision', 'average_precision', 'roc_auc'])

In [8]:
#access MLcps
gsObj_updated.cv_results_["mean_test_MLcps"]

array([2.70382103, 2.70281822, 2.69759673])

In [9]:
#access rank array based on MLcps
gsObj_updated.cv_results_["rank_test_MLcps"]

array([1, 2, 3])

## Weighted MLcps
##### Certain metrics are more significant than others in some cases. As an example, if the dataset is imbalanced, a high F1 score might be preferred to higher accuracy. A user can provide weights for metrics of interest while calculating MLcps in such a scenario. Weights should be a dictionary object where keys are metric names and values are corresponding weights. It can be passed as a parameter in getCPS.calculate() function.

In [10]:
#read input data (a dataframe) or load an example data
metrics=getCPS.sample_metrics()

#define weights
weights={"Accuracy":0.75,"F1": 1.25}

#calculate Machine Learning cumulative performance score
cpsScore=getCPS.calculate(metrics,weights)
cpsScore

,Algorithms,Score
1,Dummy,1.168150
2,GP,2.477055
3,LDA,2.507680
4,BC,2.651351
5,RF,2.672004
6,ETC,2.677232
7,SVM,2.703981
8,LR,2.714080


In [11]:
gsObj=getCPS.sample_GridSearch_Object()

#define weights
weights={"accuracy":0.75,"f1": 1.25}

#calculate Machine Learning cumulative performance score
gsObj_updated=getCPS.calculate(gsObj,weights)
gsObj_updated.cv_results_["mean_test_MLcps"]

array([2.70404499, 2.70306564, 2.69792164])